In [ ]:

import exoplanet
exoplanet.utils.docs_setup()
print(f"exoplanet.__version__ = '{exoplanet.__version__}'")


In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

url = "https://raw.githubusercontent.com/California-Planet-Search/radvel/master/example_data/epic203771098.csv"
data = pd.read_csv(url, index_col=0)

x = np.array(data.t)
y = np.array(data.vel)
yerr = np.array(data.errvel)

# Reference time and time grid
x_ref = 0.5 * (x.min() + x.max())
t = np.linspace(x.min() - 5, x.max() + 5, 1000)

# Plot data
plt.errorbar(x, y, yerr=yerr, fmt=".k")
plt.xlabel("time [days]")
plt.ylabel("radial velocity [m/s]")
plt.title("Observed Radial Velocities")
plt.grid(True)
plt.show()


In [ ]:

import exoplanet as xo

# Rough guess for period and t0 for demonstration purposes
periods = [20.8851, 42.3633]
period_errs = [0.0003, 0.0006]
t0s = [2072.7948, 2082.6251]
t0_errs = [0.0007, 0.0004]

Ks = xo.estimate_semi_amplitude(periods, x, y, yerr, t0s=t0s)
print("Estimated semi-amplitudes:", Ks, "m/s")


In [ ]:

import pymc3 as pm
import arviz as az

with pm.Model() as model:
    # Priors
    period = pm.Normal("period", mu=20.8851, sigma=0.001)
    t0 = pm.Normal("t0", mu=2072.7948, sigma=0.001)
    K = pm.Uniform("K", lower=0.0, upper=100.0)
    mu = pm.Normal("mu", mu=0.0, sigma=10.0)

    # Orbit model
    orbit = xo.orbits.KeplerianOrbit(period=period, t0=t0)

    # Radial velocity model
    rv_model = orbit.get_radial_velocity(x, K=K) + mu

    # Likelihood
    err = pm.HalfNormal("err", sigma=10)
    pm.Normal("obs", mu=rv_model, sigma=np.sqrt(yerr**2 + err**2), observed=y)

    # Sample
    trace = pm.sample(1000, tune=1000, cores=1, return_inferencedata=True)


In [ ]:

# Trace summary
az.plot_trace(trace)
plt.show()

# RV plot
period_m = trace.posterior["period"].mean().values
t0_m = trace.posterior["t0"].mean().values
K_m = trace.posterior["K"].mean().values
mu_m = trace.posterior["mu"].mean().values

orbit_m = xo.orbits.KeplerianOrbit(period=period_m, t0=t0_m)
rv_fit = orbit_m.get_radial_velocity(t, K=K_m) + mu_m

plt.errorbar(x, y, yerr=yerr, fmt=".k", label="data")
plt.plot(t, rv_fit, label="fit")
plt.xlabel("time [days]")
plt.ylabel("radial velocity [m/s]")
plt.legend()
plt.title("Radial Velocity Fit")
plt.grid(True)
plt.show()
